In [ ]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

In [ ]:
"""Load support tickets data from CSV into PostgreSQL database."""
import csv
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
from agent_tools import *


load_dotenv(os.path.join(parent_dir, ".env"))

In [ ]:
search_similar_issues("WiFi keeps dropping")

### Create the agent with tool access
We created the agent in the Azure AI Foundry project and added the Postgres tools needed to query the database.

In [ ]:
import asyncio
from random import randint
from typing import Annotated

from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent, AgentThread, ChatMessage, ChatMessageStoreProtocol
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential, DefaultAzureCredential
from pydantic import BaseModel, Field

In [ ]:
instructions = """
You are a helpful legal assistant who can retrieve information about service tickets from postgres database.
Select the following tools to answer the user's question:
- search_similar_issues(issue_description: str, limit: int = 5) -> List[Dict]
    : Searches for similar issues in the support tickets database based on the provided issue description.

Don't return the tickets not matching with user query.
"""

In [ ]:
# Initialize the agent toolset with user functions
functions = user_functions
# toolset = ToolSet()
# toolset.add(functions)

In [ ]:
async def create_agent():
    async with(
        DefaultAzureCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT"),
            api_key=os.getenv("AI_FOUNDRY_API_KEY"),
            api_version=os.getenv("AZURE_AI_API_VERSION", "2025-04-01-preview"),
            model_deployment_name=os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME_4o", "gpt-4o"),
        ) as client,
    ):
        agent = client.create_agent(
            name="TicketAnalyzer-Agent-MAF",
            instructions=instructions,
            description="An agent that can analyze and retrieve support tickets from Postgres database.",
            tools=[
                search_similar_issues,
                
            ],
            # chat_message_store_factory=CustomChatMessageStore,
        )

        return agent

In [ ]:
async def run_async(query: str, thread: AgentThread = None):
    # query = "What's the weather like in Seattle?"
    
    print(f"User: {query}")
    result = await agent.run(query, thread=thread, store=True if thread else False)
    print(f"Agent: {result}\n")
    print(f"Usage Details: {result.usage_details}\n")

    return result


In [ ]:
agent = await create_agent()
agent

In [ ]:
thread = agent.get_new_thread()
thread.message_store

In [ ]:
query = "Can you find support tickets related to login issues?"
result = await run_async(query, thread=thread)
result

In [ ]:
# dir(result.messages[0])
for msg in result.messages:
    print(dir(msg))
    print(f"{msg.role}: {msg.to_dict()}\n")